# Analysis outline

The analysis has the following components:

1. [orthologs.ipynb](orthologs.ipynb) collect primate orthologs of each human gene from NCBI, remove redundant sequences, save them to a `*.fasta` file.  

1. [align.ipynb](align.ipynb) align orthologous sequences in each `*.fasta` file and save it to a `*.aln` file.

1. [pathogen.ipynb](pathogen.ipynb) extract pathogenic variants from ClinVar data. 

1. [clean_up.ipynb](clean_up.ipynb) establish connections between transcripts in ClinVar data and proteins in ortholog collections.

1. [find_CPDs.ipynb](find_CPDs.ipynb) find putative CPDs.

# Set up

Run the following cell.  It loads some required functions.

In [1]:
run kondrashov

Run the following two cell.  The first displays the path to the current folder.  The second displays the contents of that folder.  (Note: your output will be different from mine.)

In [2]:
pwd

'/Users/arnav/hhh Dropbox/arnav mehta/Mac/Documents/GitHub/kondrashov'

In [35]:
ls

ACE2 ortholog.ipynb    clean_up.ipynb         orthologs-Copy1.ipynb
README.md              fasta/                 orthologs.ipynb
align.ipynb            find_CPDs.ipynb        pathogen.ipynb
aln/                   kondrashov.py


If there is no folder called `fasta`, create it by running the following command.

In [36]:
mkdir fasta

mkdir: fasta: File exists


If there is no folder called `aln`, create it by running the following command.

In [37]:
mkdir aln

mkdir: aln: File exists


Rerun the `ls` command above to confirm that you have created those folders successfully. 

The following defines a function that gets the species name from a NCBI sequence record.

# Get protein sequences for primate orthologs of human genes

**Note: this is a new version of the `orthologs.ipynb` notebook that prevents unique human sequences from being dropped.**

* Enter the gene name you are working on in the following cell.  In this page I'll use ALPL as an example.

## AR

In [3]:
# enter gene name here
gene = 'AR'

* In the [NCBI query page](https://www.ncbi.nlm.nih.gov/labs/gquery/) search for `Homo sapiens ALPL`.  

* When the search results come up ([this is the page for ALPL](https://www.ncbi.nlm.nih.gov/labs/gquery/all/?term=Homo+sapiens+ALPL)) click on `Orthologs`.  Here's what the output looks like for [ALPL](https://www.ncbi.nlm.nih.gov/gene/249/ortholog/?scope=9443&term=ALPL).

* Use the Taxonomy Tree to select **mammals > placentals > primates**.  For ALPL, this reduces the number of orthologs from 335 to 29.

* In the list of sequences click to select all (box labelled `0 selected`).

* Click on `Add to cart`.  The items appear in a box with a shopping cart symbol.  Click on that box.

* A popup window appears.  Click on `Protein alignment`.  Choose `all sequences per gene`.  Click `Align`.

* Select all the accession numbers in the box (XP_039330133.1, XP_037856105.1, etc) and copy them.

* Run the following cell (it pulls the names from the clipboard and puts them on a list).

In [4]:
# copy IDs from NCBI ortholog protein alignment dialog (all sequences per gene)
tmp = pd.read_clipboard(sep= '/n', names=['id'])
seq_ids = tmp['id'].tolist()
print('{0} orthologs: {1} primate sequences (retrieved on {2})\n'.format(gene, len(seq_ids), date.today()))
print(seq_ids)

AR orthologs: 57 primate sequences (retrieved on 2022-07-21)

['XP_045240204.1', 'XP_035145654.1', 'XP_024783077.2', 'XP_034805436.1', 'XP_033057282.1', 'XP_032612996.1', 'XP_032612995.1', 'XP_032612994.1', 'XP_032111718.1', 'XP_032111716.1', 'XP_032111715.1', 'XP_021788817.2', 'XP_003917866.2', 'XP_030790396.1', 'XP_030663287.1', 'XP_030663286.1', 'XP_030663285.1', 'XP_030663284.1', 'XP_025227877.1', 'XP_025227876.1', 'XP_025227875.1', 'XP_024646999.1', 'XP_024208738.1', 'XP_024096028.1', 'XP_024096027.1', 'XP_024096026.1', 'XP_023363561.1', 'XP_023054209.1', 'XP_023054208.1', 'XP_023054207.1', 'NP_001334993.1', 'NP_001334992.1', 'NP_001334990.1', 'XP_018875276.1', 'XP_017375691.1', 'XP_016798174.1', 'XP_012591725.1', 'XP_012499360.1', 'XP_012316262.1', 'XP_012316261.1', 'XP_011731144.1', 'XP_011916275.1', 'XP_011817378.1', 'XP_011835931.1', 'XP_011835925.1', 'XP_009233214.1', 'XP_008062492.1', 'XP_007990129.1', 'XP_005593867.1', 'NP_001266892.1', 'XP_003801358.1', 'XP_002762998.1', '

In [5]:
# you can do it by hand if you already have all the sequences
seq_ids = ['XP_045240204.1', 'XP_035145654.1', 'XP_024783077.2', 'XP_034805436.1', 'XP_033057282.1', 'XP_032612996.1', 'XP_032612995.1', 'XP_032612994.1', 'XP_032111718.1', 'XP_032111716.1', 'XP_032111715.1', 'XP_021788817.2', 'XP_003917866.2', 'XP_030790396.1', 'XP_030663287.1', 'XP_030663286.1', 'XP_030663285.1', 'XP_030663284.1', 'XP_025227877.1', 'XP_025227876.1', 'XP_025227875.1', 'XP_024646999.1', 'XP_024208738.1', 'XP_024096028.1', 'XP_024096027.1', 'XP_024096026.1', 'XP_023363561.1', 'XP_023054209.1', 'XP_023054208.1', 'XP_023054207.1', 'NP_001334993.1', 'NP_001334992.1', 'NP_001334990.1', 'XP_018875276.1', 'XP_017375691.1', 'XP_016798174.1', 'XP_012591725.1', 'XP_012499360.1', 'XP_012316262.1', 'XP_012316261.1', 'XP_011731144.1', 'XP_011916275.1', 'XP_011817378.1', 'XP_011835931.1', 'XP_011835925.1', 'XP_009233214.1', 'XP_008062492.1', 'XP_007990129.1', 'XP_005593867.1', 'NP_001266892.1', 'XP_003801358.1', 'XP_002762998.1', 'NP_001124649.1', 'NP_001028083.1', 'NP_001011645.1', 'NP_001009012.1', 'NP_000035.2']
print('{0} orthologs: {1} primate sequences\n'.format(gene, len(seq_ids)))
print(seq_ids)

AR orthologs: 57 primate sequences

['XP_045240204.1', 'XP_035145654.1', 'XP_024783077.2', 'XP_034805436.1', 'XP_033057282.1', 'XP_032612996.1', 'XP_032612995.1', 'XP_032612994.1', 'XP_032111718.1', 'XP_032111716.1', 'XP_032111715.1', 'XP_021788817.2', 'XP_003917866.2', 'XP_030790396.1', 'XP_030663287.1', 'XP_030663286.1', 'XP_030663285.1', 'XP_030663284.1', 'XP_025227877.1', 'XP_025227876.1', 'XP_025227875.1', 'XP_024646999.1', 'XP_024208738.1', 'XP_024096028.1', 'XP_024096027.1', 'XP_024096026.1', 'XP_023363561.1', 'XP_023054209.1', 'XP_023054208.1', 'XP_023054207.1', 'NP_001334993.1', 'NP_001334992.1', 'NP_001334990.1', 'XP_018875276.1', 'XP_017375691.1', 'XP_016798174.1', 'XP_012591725.1', 'XP_012499360.1', 'XP_012316262.1', 'XP_012316261.1', 'XP_011731144.1', 'XP_011916275.1', 'XP_011817378.1', 'XP_011835931.1', 'XP_011835925.1', 'XP_009233214.1', 'XP_008062492.1', 'XP_007990129.1', 'XP_005593867.1', 'NP_001266892.1', 'XP_003801358.1', 'XP_002762998.1', 'NP_001124649.1', 'NP_00102

* Run the next cell.  It pulls the sequences from NCBI and grabs any unique human sequences.  Then it flags any redundant sequences and drops those.  For ALPL it drops 84 sequences and we end up with 57.

In [6]:
# 1. Collect unique human sequences
print('{0} orthologs: all primate sequences\n'.format(gene))
all_sequences = []
seq_records = []
inc = 0
exc = 0
with Entrez.efetch(
    db="protein", rettype="gb", retmode="text", id=seq_ids,
) as handle:
    for seq_record in SeqIO.parse(handle, "gb"):
        sp = get_species(seq_record)
        if sp == 'Homo sapiens':
            seq = seq_record.seq
            if seq in all_sequences:
                print("\t{0}\t({1} aa)\t{2}\t*** the same as sequence {3} (excluded) ***".format(seq_record.id, len(seq), seq_record.description, all_sequences.index(seq)))
                exc += 1
            else:
                all_sequences.append(seq)
                print("{3}:\t{0}\t({1} aa)\t{2}".format(seq_record.id, len(seq), seq_record.description, inc))
                seq_records.append(seq_record)
                inc += 1
# # 2. Collect other unique sequences
with Entrez.efetch(
    db="protein", rettype="gb", retmode="text", id=seq_ids,
) as handle:
    for seq_record in SeqIO.parse(handle, "gb"):
        seq = seq_record.seq
        if seq in all_sequences:
            print("\t{0}\t({1} aa)\t{2}\t*** the same as sequence {3} (excluded) ***".format(seq_record.id, len(seq), seq_record.description, all_sequences.index(seq)))
            exc += 1
        else:
            all_sequences.append(seq)
            print("{3}:\t{0}\t({1} aa)\t{2}".format(seq_record.id, len(seq), seq_record.description, inc))
            seq_records.append(seq_record)
            inc += 1
print('\n\tTotal:\t', inc, ' unique sequences (', exc, ' excluded)', sep='')

AR orthologs: all primate sequences

0:	NP_001334993.1	(572 aa)	androgen receptor isoform 5 [Homo sapiens]
1:	NP_001334992.1	(648 aa)	androgen receptor isoform 4 [Homo sapiens]
2:	NP_001334990.1	(644 aa)	androgen receptor isoform 3 [Homo sapiens]
3:	NP_001011645.1	(388 aa)	androgen receptor isoform 2 [Homo sapiens]
4:	NP_000035.2	(920 aa)	androgen receptor isoform 1 [Homo sapiens]
5:	XP_045240204.1	(622 aa)	androgen receptor isoform X2 [Macaca fascicularis]
6:	XP_035145654.1	(627 aa)	androgen receptor isoform X2 [Callithrix jacchus]
7:	XP_024783077.2	(882 aa)	androgen receptor isoform X2 [Pan paniscus]
8:	XP_034805436.1	(907 aa)	androgen receptor isoform X1 [Pan paniscus]
9:	XP_033057282.1	(383 aa)	LOW QUALITY PROTEIN: androgen receptor [Trachypithecus francoisi]
10:	XP_032612996.1	(615 aa)	androgen receptor isoform X3 [Hylobates moloch]
11:	XP_032612995.1	(618 aa)	androgen receptor isoform X2 [Hylobates moloch]
12:	XP_032612994.1	(891 aa)	androgen receptor isoform X1 [Hylobates moloch

In [41]:
seq_records

[SeqRecord(seq=Seq('MISPFLVLAIGTCLTNSLVPEKEKDPKYWRDQAQETLKYALELQKLNTNVAKNV...VLF'), id='NP_001356734.1', name='NP_001356734', description='alkaline phosphatase, tissue-nonspecific isozyme isoform 1 preproprotein [Homo sapiens]', dbxrefs=[]),
 SeqRecord(seq=Seq('MPWSFRSSTPTWLRMSSCSWEMGQLHHNPGEETRLEMDKFPFVALSKTYNTNAQ...VLF'), id='XP_016856392.1', name='XP_016856392', description='alkaline phosphatase, tissue-nonspecific isozyme isoform X1 [Homo sapiens]', dbxrefs=['BioProject:PRJNA168']),
 SeqRecord(seq=Seq('MPWSFRSSTPTWLRMSSCSWEMTYNTNAQVPDSAGTATAYLCGVKANEGTVGVS...VLF'), id='NP_001170991.1', name='NP_001170991', description='alkaline phosphatase, tissue-nonspecific isozyme isoform 3 [Homo sapiens]', dbxrefs=[]),
 SeqRecord(seq=Seq('MFLGDGMGVSTVTAARILKGQLHHNPGEETRLEMDKFPFVALSKTYNTNAQVPD...VLF'), id='NP_001120973.2', name='NP_001120973', description='alkaline phosphatase, tissue-nonspecific isozyme isoform 2 [Homo sapiens]', dbxrefs=[]),
 SeqRecord(seq=Seq('MISPFLVLAIGTCLTNSLVPEKEKDPKYWRDQ

* Copy the output to the cell below

* The following cell writes the sequences to the file `ALPL.fasta` (fasta format).

In [7]:
# write the sequences to a fasta file
print('{0} orthologs: unique primate sequences\n'.format(gene))
i = 0
with open("fasta/{0}.fasta".format(gene), "w") as output:
    for seq_record in seq_records:
        print("{3}:\t{0}\t({1} aa)\t{2}".format(seq_record.id, len(seq), seq_record.description, i))
        SeqIO.write(seq_record, output, "fasta")
        i += 1
print("\n{0}.fasta saved!".format(gene))

AR orthologs: unique primate sequences

0:	NP_001334993.1	(920 aa)	androgen receptor isoform 5 [Homo sapiens]
1:	NP_001334992.1	(920 aa)	androgen receptor isoform 4 [Homo sapiens]
2:	NP_001334990.1	(920 aa)	androgen receptor isoform 3 [Homo sapiens]
3:	NP_001011645.1	(920 aa)	androgen receptor isoform 2 [Homo sapiens]
4:	NP_000035.2	(920 aa)	androgen receptor isoform 1 [Homo sapiens]
5:	XP_045240204.1	(920 aa)	androgen receptor isoform X2 [Macaca fascicularis]
6:	XP_035145654.1	(920 aa)	androgen receptor isoform X2 [Callithrix jacchus]
7:	XP_024783077.2	(920 aa)	androgen receptor isoform X2 [Pan paniscus]
8:	XP_034805436.1	(920 aa)	androgen receptor isoform X1 [Pan paniscus]
9:	XP_033057282.1	(920 aa)	LOW QUALITY PROTEIN: androgen receptor [Trachypithecus francoisi]
10:	XP_032612996.1	(920 aa)	androgen receptor isoform X3 [Hylobates moloch]
11:	XP_032612995.1	(920 aa)	androgen receptor isoform X2 [Hylobates moloch]
12:	XP_032612994.1	(920 aa)	androgen receptor isoform X1 [Hylobates mol

* The following cell aligns the sequences and writes them to the file `ALPL.aln` (also in fasta format).  To look at the alignment open it in [this viewer application](https://alignmentviewer.org/).  (Hannah: you may not be able to run this step.  Just skip it and repeat the sequence processing above for the other genes.)

In [8]:
# align sequences
print('{0} orthologs\n'.format(gene))
muscle_cline = MuscleCommandline(input='fasta/{0}.fasta'.format(gene))
stdout, stderr = muscle_cline()
align = AlignIO.read(StringIO(stdout), 'fasta')
AlignIO.write(align, 'aln/{0}.aln'.format(gene), 'fasta')
print(align)
print("\n{0}.aln saved!".format(gene))

AR orthologs



ApplicationError: Non-zero return code 127 from 'muscle -in fasta/AR.fasta', message '/bin/sh: muscle: command not found'

* Repeat for other genes.

In [10]:
!open .